1. 환경설정 및 필요한 라이브러리 설치  

In [1]:
# 환경 설정 및 필요한 라이브러리 설치
!pip install tensorflow tensorflow_hub tf2onnx openvino-dev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 9.7 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.0+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.3.0+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which 

2. Google Drive 마운트  

In [2]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


3. 로컬 데이터셋 준비  

In [3]:
# 데이터셋 준비
import tensorflow as tf
import tensorflow_hub as hub
import os
import numpy as np
import pickle  # pickle 임포트 추가

# 데이터셋 경로 설정 (Google Drive에 있는 경로)
train_dir = '/content/drive/MyDrive/Datas/train'  # 'train' 폴더 경로
validation_dir = '/content/drive/MyDrive/Datas/valid'  # 'validation' 폴더 경로

# 클래스 레이블 추출 및 저장 (한 번 실행 후 주석 처리 가능)
class_names = sorted(os.listdir(train_dir))
with open('/content/drive/MyDrive/class_names.pkl', 'wb') as f:
    pickle.dump(class_names, f)

# 클래스 레이블 로드
with open('/content/drive/MyDrive/class_names.pkl', 'rb') as f:
    class_names = pickle.load(f)

# 데이터 전처리 함수
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224)) / 255.0
    return image, label

# 이미지 로드 및 전처리 함수
def load_and_preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    return preprocess(image, tf.constant(0))

# 데이터셋 생성 함수
def create_dataset(directory):
    dataset = tf.data.Dataset.list_files(os.path.join(directory, '*/*'))
    dataset = dataset.map(lambda x: load_and_preprocess_image(x))
    return dataset

# 데이터셋 크기 계산
train_dataset_size = len(tf.io.gfile.glob(train_dir + '/*/*'))
validation_dataset_size = len(tf.io.gfile.glob(validation_dir + '/*/*'))

# 로컬 데이터셋 로드 및 배치 처리
batch_size = 32
buffer_size = train_dataset_size  # 셔플 버퍼 크기를 train 데이터셋의 크기로 설정
ds_train = create_dataset(train_dir).shuffle(buffer_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = create_dataset(validation_dir).shuffle(buffer_size).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


# 로컬 데이터셋 로드 및 배치 처리
batch_size = 32
ds_train = create_dataset(train_dir).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = create_dataset(validation_dir).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)


4. 모델 로드 및 커스터마이징  

In [4]:
# 모델 로드 및 커스터마이징
model_url = "https://tfhub.dev/tensorflow/efficientnet/b0/classification/1"
feature_extractor_layer = hub.KerasLayer(model_url, input_shape=(224, 224, 3), trainable=False)

model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(len(class_names), activation='softmax')  # 데이터셋에 맞는 출력 레이어 (클래스 수에 맞게 조정)
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


5. 모델 훈련  

In [ ]:
# 모델 훈련
epochs = 5
history = model.fit(ds_train, validation_data=ds_val, epochs=epochs)


Epoch 1/5
61/61 [==============================] - 550s 9s/step - loss: 1.9763 - accuracy: 0.9241 - val_loss: 1.8737 - val_accuracy: 1.0000
Epoch 2/5
61/61 [==============================] - 377s 6s/step - loss: 1.7791 - accuracy: 1.0000 - val_loss: 1.6821 - val_accuracy: 1.0000
Epoch 3/5
29/61 [=============>................] - ETA: 2:41 - loss: 1.6394 - accuracy: 1.0000

6. ONNX로 모델 변환  

In [14]:
# ONNX로 모델 변환
import tf2onnx
spec = (tf.TensorSpec((None, 224, 224, 3), tf.float32),)
output_path = "efficientnet_b0.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
with open(output_path, "wb") as f:
    f.write(model_proto.SerializeToString())

7. OpenVINO로 모델 최적화 및 추론  

In [15]:
# OpenVINO로 모델 최적화 및 추론
from openvino.runtime import Core

ie = Core()
model_onnx_path = "/content/efficientnet_b0.onnx"
model = ie.read_model(model=model_onnx_path)
compiled_model = ie.compile_model(model=model, device_name="CPU")

input_layer = compiled_model.input(0)
output_layer = compiled_model.output(0)

# 이미지 전처리 함수
def preprocess_image_openvino(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)  # 배치 차원 추가 (N, H, W, C)
    return img

# OpenVINO 예측 함수
def predict_openvino(image_path):
    img = preprocess_image_openvino(image_path)
    predictions = compiled_model([img])
    predicted_class = np.argmax(predictions[output_layer], axis=-1)
    return class_names[predicted_class[0]]  # 예측된 클래스 이름 반환

# 테스트 이미지 예측
image_path = "/content/drive/MyDrive/Datas/test.jpg"  # 여기에 테스트할 이미지 경로를 입력하세요.
predicted_class = predict_openvino(image_path)
print(f"Predicted class: {predicted_class}")

Predicted class: Bonobono


8. ONNX 모델을 OpenVINO IR 형식으로 변환 \
위의 명령어를 통해 ONNX 모델을 OpenVINO IR 형식으로 변환할 수 있습니다. 변환된 .xml 및 .bin 파일은 지정된 출력 디렉토리(/content/drive/MyDrive/efficientnet_model)에 저장됩니다.

In [16]:

# 모델 최적화를 위해 OpenVINO의 Model Optimizer 사용
!mo --input_model efficientnet_b0.onnx --output_dir /content/drive/MyDrive/efficientnet_model

# 출력 디렉토리의 파일 목록 확인
!ls /content/drive/MyDrive/efficientnet_model

[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html
[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release. Please use OpenVINO Model Converter (OVC). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /content/drive/MyDrive/efficientnet_model/efficientnet_b0.xml
[ SUCCESS ] BIN file: /content/drive/MyDrive/efficientnet_model/efficientnet_b0.bin
efficientnet_b0.bin  efficientnet_b0.xml
